# Data Sources

In [72]:
import requests 
import pandas as pd
import json
from datetime import datetime, timedelta
import math
from itertools import product
import io
import oracledb
import os

os.environ["TNS_ADMIN"] = r"C:\Users\addel\Downloads\Wallet_DS320Flights\cwallet.sso"


In [46]:
with open('keys.json', 'r') as KeysFile:
    data = json.load(KeysFile)

aviationStackAPIKey = data["Aviation Stack API KEY"]
username = "admin"
password = data["Oracle Password"]
dsn = data["DB Name"]

## Country Data

In [7]:
def getCountryCodes(country: str):
    urlCountry = f"https://api.aviationstack.com/v1/countries?access_key={aviationStackAPIKey}"
    queryStringCountry = {"search": country}
    responseCountry = requests.get(urlCountry, params=queryStringCountry)
    responseCountry = responseCountry.json()
    responseCountry = responseCountry.get('data')
    codes = [c.get('country_iso2') for c in responseCountry]
    return codes

In [8]:
def getAirportsByCountry(code: str):
    urlAirport = f"https://api.aviationstack.com/v1/airports?access_key={aviationStackAPIKey}"

    offset = 0
    all_data = []
    
    while True:
        queryStringAirport = {"offset": offset, "country_iso": code, "type": "international"}
        responseAirport = requests.get(urlAirport, params=queryStringAirport)
        responseAirport = responseAirport.json()
        
        currentData = responseAirport.get('data', [])
        currentData = [airP for airP in currentData if airP.get('country_iso2') == code]
        all_data.extend(currentData)
        
        if offset + 100 >= responseAirport.get('pagination').get('total'):
            break
        
        offset += 100
    
    airports_df = pd.DataFrame(columns=['IATA', 'CountryCode', 'AirportName', 'Country'])
    
    for case in all_data:
        new_case = {'IATA': case.get('iata_code', ''),
                    'CountryCode': case.get('country_iso2', ''),
                    'AirportName': case.get('airport_name', ''),
                    'Country': case.get('country_name', '')}
        airports_df.loc[len(airports_df)] = new_case
        
    return airports_df

## Historical Flight Data

In [9]:
def getHistoricalFlights(departureIATA: str=None, arrivalIATA: str=None, date: str=None):
    url = f"https://api.aviationstack.com/v1/flights?access_key={aviationStackAPIKey}"
    queryString = {"limit": 100, "dep_iata": departureIATA, "arr_iata": arrivalIATA, "flight_date": date}
    queryString = {key: value for key, value in queryString.items() if value is not None}
    print(queryString)
    response = requests.get(url, params=queryString)
    if response.status_code != 200:
        print(response)
        return None

    response = response.json()
    response = response['data']
    flights_df = pd.DataFrame(columns=['Date', 'DepartureIATA', 'DepartureTime', 'ArrivalIATA', 'ArrivalTime', 'Aircraft', 'AirlineName', 'AirlineIATA', 'FlightNumberIATA'])
    for flight in response:
        if isinstance(flight, dict) and flight.get('flight', {}).get('codeshared') is None:
            new_case = {
                'Date': date,
                'DepartureIATA': flight.get('departure', {}).get('iata', None),
                'DepartureTime': flight.get('departure', {}).get('scheduled', None),
                'ArrivalIATA': flight.get('arrival', {}).get('iata', None),
                'ArrivalTime': flight.get('arrival', {}).get('scheduled', None),
                'Aircraft': flight.get('aircraft', {}).get('iata', None) if flight.get('aircraft') is not None else None,
                'AirlineName': flight.get('airline', {}).get('name', None),
                'AirlineIATA': flight.get('airline', {}).get('iata', None),
                'FlightNumberIATA': flight.get('flight', {}).get('iata', None)
            }
            flights_df.loc[len(flights_df)] = new_case

    return flights_df

### Get Country Codes for US Brazil Portugal Canada Italy France

In [10]:
US_codes = getCountryCodes('United States')
Brazil_codes = getCountryCodes('Brazil')
Portugal_codes = getCountryCodes('Portugal')
Mexico_codes = getCountryCodes('Mexico')
Italy_codes = getCountryCodes('Italy')
France_codes = getCountryCodes('France')

In [11]:
print(f"Country codes:\nUS: {US_codes}\nBrazil: {Brazil_codes}\nPortugal: {Portugal_codes}\nMexico: {Mexico_codes}\nItaly: {Italy_codes}\nFrance: {France_codes}")

Country codes:
US: ['UM', 'US']
Brazil: ['BR']
Portugal: ['PT']
Mexico: ['MX']
Italy: ['IT']
France: ['FR', 'MQ']


In [12]:
US_airports = ['JFK', 'ATL', 'DTW', 'LAX']
Italy_airports = ['FCO']
France_airports = ['CDG']
Brazil_airports = ['GRU']
Portugal_airports = ['LIS']
Mexico_airports = ['CUN']

In [13]:
US_Italy = list(product(US_airports, Italy_airports))
US_France = list(product(US_airports, France_airports))
US_Brazil = list(product(US_airports, Brazil_airports))
US_Portugal = list(product(US_airports, Portugal_airports))
US_Mexico = list(product(US_airports, Mexico_airports))
airport_combinations = US_Italy + US_France + US_Brazil + US_Portugal + US_Mexico

In [30]:
dates = [datetime(2024, 1, 1) + timedelta(days=i) for i in range(366)]
dates = [i.strftime("%Y-%m-%d") for i in dates]

In [ ]:
df_all_flights = pd.DataFrame(columns=['Date', 'DepartureIATA', 'DepartureTime', 'ArrivalIATA', 'ArrivalTime', 'Aircraft', 'AirlineName', 'AirlineIATA', 'FlightNumberIATA'])
for city_pair in airport_combinations:
    for current_date in dates:
        temp_df = getHistoricalFlights(departureIATA=city_pair[0], arrivalIATA=city_pair[1], date=current_date)
        df_all_flights = pd.concat([temp_df, df_all_flights], ignore_index=True)

In [ ]:
df_all_flights.head()

,Date,DepartureIATA,DepartureTime,ArrivalIATA,ArrivalTime,Aircraft,AirlineName,AirlineIATA,FlightNumberIATA
0,2024-11-22,ATL,2024-11-22T02:00:00+00:00,LIS,2024-11-22T15:00:00+00:00,B764,Delta Air Lines,DL,DL272
1,2024-12-11,JFK,2024-12-11T20:04:00+00:00,LIS,2024-12-12T08:00:00+00:00,None,Delta Air Lines,DL,DL272
2,2024-12-11,JFK,2024-12-11T22:00:00+00:00,LIS,2024-12-12T09:55:00+00:00,None,TAP Air Portugal,TP,TP210
3,2024-12-10,JFK,2024-12-10T20:04:00+00:00,LIS,2024-12-11T08:00:00+00:00,B764,Delta Air Lines,DL,DL272
4,2024-12-10,JFK,2024-12-10T22:00:00+00:00,LIS,2024-12-11T09:55:00+00:00,A339,TAP Air Portugal,TP,TP210


In [33]:
df_all_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20169 entries, 0 to 20168
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              20169 non-null  object
 1   DepartureIATA     20169 non-null  object
 2   DepartureTime     20169 non-null  object
 3   ArrivalIATA       20169 non-null  object
 4   ArrivalTime       20169 non-null  object
 5   Aircraft          17037 non-null  object
 6   AirlineName       20169 non-null  object
 7   AirlineIATA       20150 non-null  object
 8   FlightNumberIATA  20150 non-null  object
dtypes: object(9)
memory usage: 1.4+ MB


In [ ]:
df_all_flights['Date'] = pd.to_datetime(df_all_flights['Date'])
df_all_flights['DepartureTime'] = pd.to_datetime(df_all_flights['DepartureTime'])
df_all_flights['ArrivalTime'] = pd.to_datetime(df_all_flights['ArrivalTime'])

In [37]:
df_all_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20169 entries, 0 to 20168
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Date              20169 non-null  datetime64[ns]     
 1   DepartureIATA     20169 non-null  object             
 2   DepartureTime     20169 non-null  datetime64[ns, UTC]
 3   ArrivalIATA       20169 non-null  object             
 4   ArrivalTime       20169 non-null  datetime64[ns, UTC]
 5   Aircraft          17037 non-null  object             
 6   AirlineName       20169 non-null  object             
 7   AirlineIATA       20150 non-null  object             
 8   FlightNumberIATA  20150 non-null  object             
dtypes: datetime64[ns, UTC](2), datetime64[ns](1), object(6)
memory usage: 1.4+ MB


In [38]:
df_all_flights[df_all_flights['FlightNumberIATA'].isna()]

,Date,DepartureIATA,DepartureTime,ArrivalIATA,ArrivalTime,Aircraft,AirlineName,AirlineIATA,FlightNumberIATA
1007,2024-04-19,LAX,2024-04-19 09:00:00+00:00,CUN,2024-04-19 15:28:00+00:00,None,empty,None,None
1314,2024-02-20,LAX,2024-02-20 10:35:00+00:00,CUN,2024-02-20 17:31:00+00:00,None,empty,None,None
3610,2024-06-15,ATL,2024-06-15 06:15:00+00:00,CUN,2024-06-15 07:33:00+00:00,None,Vista America,None,None
4398,2024-03-01,ATL,2024-03-01 13:30:00+00:00,CUN,2024-03-01 16:02:00+00:00,None,empty,None,None
5134,2024-10-29,JFK,2024-10-29 18:15:00+00:00,CUN,2024-10-29 20:58:00+00:00,None,Aerolineas Ejecutivas,None,None
7011,2024-11-05,JFK,2024-11-05 14:30:00+00:00,LIS,2024-11-06 01:43:00+00:00,None,FAI rent-a-jet,None,None
7096,2024-09-25,JFK,2024-09-25 21:00:00+00:00,LIS,2024-09-26 07:59:00+00:00,None,empty,None,None
7123,2024-09-12,JFK,2024-09-12 13:45:00+00:00,LIS,2024-09-13 01:11:00+00:00,None,empty,None,None
7935,2024-09-17,ATL,2024-09-17 09:30:00+00:00,GRU,2024-09-17 19:37:00+00:00,None,empty,None,None
8164,2024-05-21,ATL,2024-05-21 10:00:00+00:00,GRU,2024-05-21 20:12:00+00:00,None,empty,None,None


In [39]:
df_all_flights['FlightNumberIATA'] = df_all_flights['FlightNumberIATA'].fillna("N/A")

In [41]:
df_all_flights.to_csv("AllFlights.csv")

In [42]:
df_all_flights.head()

,Date,DepartureIATA,DepartureTime,ArrivalIATA,ArrivalTime,Aircraft,AirlineName,AirlineIATA,FlightNumberIATA
0,2024-12-11,LAX,2024-12-11 10:36:00+00:00,CUN,2024-12-11 18:19:00+00:00,B39M,Alaska Airlines,AS,AS1332
1,2024-12-11,LAX,2024-12-11 07:14:00+00:00,CUN,2024-12-11 15:00:00+00:00,B39M,Alaska Airlines,AS,AS1366
2,2024-12-11,LAX,2024-12-11 10:35:00+00:00,CUN,2024-12-11 18:15:00+00:00,B739,Delta Air Lines,DL,DL615
3,2024-12-11,LAX,2024-12-11 09:00:00+00:00,CUN,2024-12-11 16:45:00+00:00,B39M,United Airlines,UA,UA1276
4,2024-12-10,LAX,2024-12-10 10:36:00+00:00,CUN,2024-12-10 18:19:00+00:00,B39M,Alaska Airlines,AS,AS1332


## Connect to Oracle

In [101]:
df_all_flights = pd.read_csv('AllFlights.csv').drop(columns=['Unnamed: 0'])
df_all_flights['DepartureTime'] = pd.to_datetime(df_all_flights['DepartureTime']).dt.tz_localize(None)
df_all_flights['ArrivalTime'] = pd.to_datetime(df_all_flights['ArrivalTime']).dt.tz_localize(None)


In [107]:
df_all_flights.to_csv("AllFlights.csv")

In [58]:
connection = oracledb.connect(user=username, password=password, dsn=dsn)
print(f"Database Version: {connection.version}")

Database Version: 23.6.0.24.11


In [ ]:
# Create Flights Table
sqlCreateFlights = """
    CREATE TABLE FLIGHTS(
        FlightDate DATE,
        DepartureIATA VARCHAR(3),
        DepartureTime TIMESTAMP,
        ArrivalIATA VARCHAR(3),
        ArrivalTime TIMESTAMP,
        Aircraft VARCHAR(10),
        AirlineName VARCHAR(50),
        AirlineIATA VARCHAR(3),
        FlightNumberIATA VARCHAR(30),
        PRIMARY KEY (FlightDate, FlightNumberIATA));
"""
with connection.cursor() as cursor:
    cursor.execute(sqlCreateFlights)

connection.commit()

In [99]:
df_all_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20169 entries, 0 to 20168
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Date              20169 non-null  object
 1   DepartureIATA     20169 non-null  object
 2   DepartureTime     20169 non-null  object
 3   ArrivalIATA       20169 non-null  object
 4   ArrivalTime       20169 non-null  object
 5   Aircraft          17037 non-null  object
 6   AirlineName       20169 non-null  object
 7   AirlineIATA       20150 non-null  object
 8   FlightNumberIATA  20150 non-null  object
dtypes: object(9)
memory usage: 1.4+ MB


In [ ]:
# Insert data into oracle
insert_statements = []
for row in df_all_flights.itertuples():
    qryInsert = f"INSERT INTO FLIGHTS VALUES('{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}', '{row[7]}', '{row[8]}', '{row[9]}')"
    insert_statements.append(qryInsert)

insert_statements = " ".join(insert_statements)

INSERT INTO FLIGHTS VALUES('2024-12-11', 'LAX', '2024-12-11 10:36:00+00:00', 'CUN', '2024-12-11 18:19:00+00:00', 'B39M', 'Alaska Airlines', 'AS', 'AS1332') INSERT INTO FLIGHTS VALUES('2024-12-11', 'LAX', '2024-12-11 07:14:00+00:00', 'CUN', '2024-12-11 15:00:00+00:00', 'B39M', 'Alaska Airlines', 'AS', 'AS1366') INSERT INTO FLIGHTS VALUES('2024-12-11', 'LAX', '2024-12-11 10:35:00+00:00', 'CUN', '2024-12-11 18:15:00+00:00', 'B739', 'Delta Air Lines', 'DL', 'DL615') INSERT INTO FLIGHTS VALUES('2024-12-11', 'LAX', '2024-12-11 09:00:00+00:00', 'CUN', '2024-12-11 16:45:00+00:00', 'B39M', 'United Airlines', 'UA', 'UA1276') INSERT INTO FLIGHTS VALUES('2024-12-10', 'LAX', '2024-12-10 10:36:00+00:00', 'CUN', '2024-12-10 18:19:00+00:00', 'B39M', 'Alaska Airlines', 'AS', 'AS1332') INSERT INTO FLIGHTS VALUES('2024-12-10', 'LAX', '2024-12-10 07:14:00+00:00', 'CUN', '2024-12-10 15:00:00+00:00', 'B39M', 'Alaska Airlines', 'AS', 'AS1366') INSERT INTO FLIGHTS VALUES('2024-12-10', 'LAX', '2024-12-10 10:35

In [93]:
with connection.cursor() as cursor:
    cursor.execute(insert_statements)
    cursor.commit()

DatabaseError: DPY-4011: the database or network closed the connection
Help: https://python-oracledb.readthedocs.io/en/latest/user_guide/troubleshooting.html#dpy-4011

In [ ]:
connection.commit()

In [68]:
df_econ = pd.read_csv('economic_indicators.csv')
df_econ.head()

,Reference area,TIME_PERIOD,Production,Unemployment,Confidence,Baro
0,France,2024-09,98.350045,7.5,-26.6,0.200768
1,France,2024-08,99.105151,7.5,-27.7,0.356471
2,France,2024-07,98.202947,7.5,-38.5,-0.123837
3,France,2024-06,97.898943,7.5,-32.6,-0.123683
4,France,2024-05,97.163451,7.5,-30.4,0.279064


In [66]:
sqlCreateEcon = """ 
    CREATE TABLE ECON_DATA(
        COUNTRY VARCHAR(30), 
        TIME_PERIOD VARCHAR(30),
        PRODUCTION FLOAT, 
        UNEMPLOYMENT FLOAT,
        CONFIDENCE FLOAT,
        BARRO FLOAT,
        PRIMARY KEY(COUNTRY, TIME_PERIOD)
    )
"""
with connection.cursor() as cursor:
    cursor.execute(sqlCreateEcon)

connection.commit()

In [ ]:
# Insert data into oracle
with connection.cursor() as cursor:
    for row in df_econ.itertuples():
        qryInsert = f"INSERT INTO ECON_DATA VALUES('{row[1]}', '{row[2]}', '{row[3]}', '{row[4]}', '{row[5]}', '{row[6]}')"
        print(qryInsert)
        # cursor.execute(qryInsert)

connection.commit()

## OECD API

https://data-explorer.oecd.org/vis?fs[0]=Topic%2C1%7CEconomy%23ECO%23%7CShort-term%20economic%20statistics%23ECO_STS%23&pg=0&fc=Topic&bp=true&snb=54&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_KEI%40DF_KEI&df[ag]=OECD.SDD.STES&df[vs]=4.0&dq=.M.PRVM.IX.BTE..&lom=LASTNPERIODS&lo=5&to[TIME_PERIOD]=false&ly[cl]=TIME_PERIOD&ly[rw]=REF_AREA

https://www.oecd.org/en/data/insights/data-explainers/2024/09/api.html

In [139]:
urlConsumerConfidence= 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_CS,4.0/.M.ES+IN+CCICP......?startPeriod=2024-07'

In [134]:
urlKeyEconomic = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_KEI@DF_KEI,4.0/.M.PRVM.IX.BTE..?startPeriod=2024-01&endPeriod=2024-12&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
responseKeyEconomic = requests.get(urlKeyEconomic).content
urlUnemployment = 'https://sdmx.oecd.org/public/rest/data/OECD.SDD.TPS,DSD_LFS@DF_IALFS_UNE_M,1.0/..._Z.Y._T.Y_GE15..M?startPeriod=2024-01&endPeriod=2024-12&dimensionAtObservation=AllDimensions&format=csvfilewithlabels'
responseUnemployment = requests.get(urlUnemployment).content

In [135]:
df_KeyEconomic = pd.read_csv(io.StringIO(responseKeyEconomic.decode('utf-8')))
df_Unemployment = pd.read_csv(io.StringIO(responseUnemployment.decode('utf-8')))

In [136]:
df_Unemployment.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,MEASURE,Measure,UNIT_MEASURE,Unit of measure,...,OBS_VALUE,Observation value,BASE_PER,Base period,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals
0,DATAFLOW,OECD.SDD.TPS:DSD_LFS@DF_IALFS_UNE_M(1.0),Monthly unemployment rates,I,GRC,Greece,UNE_LF_M,Monthly unemployment rate,PT_LF_SUB,Percentage of labour force in the same subgroup,...,11.200000,NaN,NaN,NaN,A,Normal value,0,Units,1,One
1,DATAFLOW,OECD.SDD.TPS:DSD_LFS@DF_IALFS_UNE_M(1.0),Monthly unemployment rates,I,GRC,Greece,UNE_LF_M,Monthly unemployment rate,PT_LF_SUB,Percentage of labour force in the same subgroup,...,9.800000,NaN,NaN,NaN,A,Normal value,0,Units,1,One
2,DATAFLOW,OECD.SDD.TPS:DSD_LFS@DF_IALFS_UNE_M(1.0),Monthly unemployment rates,I,OECD,OECD,UNE_LF_M,Monthly unemployment rate,PT_LF_SUB,Percentage of labour force in the same subgroup,...,4.901197,NaN,NaN,NaN,E,Estimated value,0,Units,1,One
3,DATAFLOW,OECD.SDD.TPS:DSD_LFS@DF_IALFS_UNE_M(1.0),Monthly unemployment rates,I,OECD,OECD,UNE_LF_M,Monthly unemployment rate,PT_LF_SUB,Percentage of labour force in the same subgroup,...,4.876813,NaN,NaN,NaN,E,Estimated value,0,Units,1,One
4,DATAFLOW,OECD.SDD.TPS:DSD_LFS@DF_IALFS_UNE_M(1.0),Monthly unemployment rates,I,OECD,OECD,UNE_LF_M,Monthly unemployment rate,PT_LF_SUB,Percentage of labour force in the same subgroup,...,4.932102,NaN,NaN,NaN,E,Estimated value,0,Units,1,One


In [137]:
df_KeyEconomic = df_KeyEconomic[df_KeyEconomic['Reference area'].isin(['Italy', 'France', 'Mexico', 'Brazil', 'Portugal'])]
df_Unemployment = df_Unemployment[df_Unemployment['Reference area'].isin(['Italy', 'France', 'Mexico', 'Brazil', 'Portugal'])]

In [138]:
df_Unemployment[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']].sort_values('Reference area')

,Reference area,TIME_PERIOD,OBS_VALUE
409,France,2024-09,7.500000
401,France,2024-07,7.500000
402,France,2024-06,7.500000
403,France,2024-05,7.500000
383,France,2024-08,7.500000
405,France,2024-03,7.400000
406,France,2024-02,7.500000
407,France,2024-01,7.500000
404,France,2024-04,7.400000
408,France,2024-10,7.600000


In [ ]:
df_KeyEconomic[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']].sort_values('Reference area')

,Reference area,TIME_PERIOD,OBS_VALUE
345,Brazil,2024-09,99.233133
288,Brazil,2024-07,97.884998
289,Brazil,2024-06,99.213821
290,Brazil,2024-05,95.048688
291,Brazil,2024-04,96.425952
292,Brazil,2024-03,96.743622
293,Brazil,2024-02,95.904895
294,Brazil,2024-01,95.721254
287,Brazil,2024-08,98.129663
321,France,2024-10,98.222560
